In [1]:
import numpy as np
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, ELU
from keras import backend as K
from keras.layers.core import Flatten, Dense, Dropout, Lambda
import pickle
from scipy import misc
from scipy.misc import imresize
from PIL import Image
import glob, os
import cv2
import csv
import math
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.layers import Convolution2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
import json

Using TensorFlow backend.


In [2]:
learning_rate = 0.0001
batch_size = 128
nb_epoch = 5

In [3]:
training_file = "train.p"
testing_file = "test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
    #print(train)

with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    #print (test)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

print (X_train.shape)
print (y_train.shape)
    


(39209, 32, 32, 3)
(39209,)


In [4]:
X_train = np.resize(X_train,(39209,66,200,3))
input_shape = X_train.shape
print (input_shape)

(39209, 66, 200, 3)


In [5]:
model = Sequential()
model.add(Lambda(lambda x:x/127.5 -1., input_shape = (66,200,3)))
model.add(Convolution2D(24, 5, 5,border_mode='same',subsample=(2,2), dim_ordering='tf', input_shape=(66,200,3)))
model.add(ELU())
model.add(Convolution2D(36, 5, 5, border_mode='same',subsample=(2,2)))
model.add(ELU())
model.add(Convolution2D(48, 5, 5,subsample=(2,2)))
model.add(ELU())
model.add(Convolution2D(64, 3, 3,subsample=(2,2)))
model.add(ELU())
model.add(Convolution2D(64, 3, 3,subsample=(2,2)))
model.add(Flatten())
model.add(ELU())
model.add(Dense(1164))
model.add(ELU())
model.add(Dense(100))
model.add(ELU())
model.add(Dense(50))
model.add(ELU())
model.add(Dense(10))
model.add(ELU())
model.add(Dense(1,name='output'))


In [7]:
def normalize(image_data):
    """
    Normalize the image data with Min-Max scaling to a range of [-0.5, 0.5]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    #costants
    a = -0.5
    b = 0.5
    xmin = np.min(image_data)
    xmax = np.max(image_data) 
    
    x = image_data    
    x_prime = a + ((x-xmin)*(b-a))/(xmax-xmin)
    
    return x_prime



In [10]:
optimizer = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, validation_split=0.3, shuffle=True)

Train on 27446 samples, validate on 11763 samples
Epoch 1/5
27446/27446 [==============================] - 296s - loss: 41.7685 - acc: 0.0525 - val_loss: 597.8218 - val_acc: 0.0000e+00
Epoch 2/5
27446/27446 [==============================] - 286s - loss: 30.6335 - acc: 0.0535 - val_loss: 546.3282 - val_acc: 0.0000e+00
Epoch 3/5
27446/27446 [==============================] - 287s - loss: 30.6970 - acc: 0.0540 - val_loss: 577.4276 - val_acc: 0.0000e+00
Epoch 4/5
27446/27446 [==============================] - 290s - loss: 30.6386 - acc: 0.0513 - val_loss: 556.9162 - val_acc: 0.0000e+00
Epoch 5/5
27446/27446 [==============================] - 290s - loss: 30.6207 - acc: 0.0532 - val_loss: 586.3330 - val_acc: 0.0000e+00


In [11]:
print(history.history['acc'][-1])

0.053231800627


In [12]:
json_string = model.to_json()
with open('model.json','w') as f:
    json.dump(json_string,f,ensure_ascii=False)

model.save_weights('model.h5')